# Pigs Style Game Competition

## Educators Note - Rational for this guided student task.

The plan !!  Get the students to do some strategies for the "intro to python" course they get in term 1. I write a shiny app that they submit the strategies to the "competion" code and the overall winner gets a prize.

In term 2 for STOR-609, they write the "competition" code (as badly as possible), swap solutions, and rewrite wrt the 5Rs.

I then introduce "patterns". They then improve there own code for th strategies and the competition from previous exercise using patterns, present it, and report on how the 5Rs  have been improved by use of patterns and other things. 

## Strategy Type

$Strategy : N \times {N} \times N \times N \rightarrow Bool$

### Example 1 - stick at 10

In [1]:
def stick_at_10(target,score,opponent_score,running_total) :
    return True if running_total < 10 else False 

### Example 2 - stick at 15

In [2]:
def stick_at_15(target,score,opponent_score,running_total) :
    return True if running_total < 15 else False 

### Example 3 - a parameterised strategy - stick at n 

In [45]:
def stick_at_n(score,opponent_score,running_total,target,n) :
    return True if running_total < n else False 

Note that the above has type $N \times N \times N \times N \times N \rightarrow Bool$ so is not of type $Strategy$. However, currying (a standard design pattern) of the strategies parameter n results in the required $Strategy$ type _i.e_. $N \times N \times N \times N \rightarrow Bool$

## Basic Turn

$Turn : Strategy \times N \times N \rightarrow N$

In [4]:
import random

In [5]:
def basic_pigs_turn(a,ascore,bscore) :
    running_total = 0
    while a(ascore,bscore,running_total) :
        roll = random.randint(1,6)
        if roll == 6 :
            return ascore
        running_total += roll
    return running_total 

### Educators Note - point of departure

Clearly, the fixed parameter values in this method shout out for abstraction, perhaps via a dice type (?). However, a big education point here is the random, and hence non-pure, nature of the function. This can be a "point of departure". Dont forget to point out the failure of memoisation of a non pure function. Also - see the idea sketched out below. 

#### Point of departure

An obvious abstraction is 

$Dice : () \rightarrow N$

which could lead to a parameterised dice function suction as 

In [6]:
def dice(sides) :
    return random.randint(1,sides)

All well and good, but the next step is the non-pure nature of this type (remember, a function with no arguments is a constant). Now get students to consider

$Dice : N  \rightarrow N \times N$

and the function

In [7]:
import sys

def dice(seed,sides) :
    random.seed(seed)
    return (random.randint(0,sys.maxsize),random.randint(1,sides))

In [8]:
seed = 0
(seed,value) = dice(seed,6)

In [11]:
from functools import partial
seed = 0
D6 = partial(dice,sides = 6)
(seed,value) = D6(seed)

This approach still has a number of issues - but it sets sail a whole ship of discovery and discussion. If time / ability permit, get the students to study this example ...

In [12]:
def diceClass(sides,seed) : # remember - a class is just function that returns an object
    def dice() : # the object produced by the class - remember, an object is just a function with memory
        nonlocal seed # the objects memory
        random.seed(seed)
        seed = random.randint(0,sys.maxsize) # note - python 3 specific
        dice,_ = diceClass(sides,seed) # notice the reference to the enclosing function
        return dice,random.randint(1,sides)
    return dice,_
        

In [13]:
D6,_ = diceClass(6,0)
D10,_ = diceClass(6,5)

In [14]:
for i in range(10) :
    D6,value = D6()
    print("D6 gives : " + str(value))

D6 gives : 4
D6 gives : 6
D6 gives : 2
D6 gives : 4
D6 gives : 6
D6 gives : 3
D6 gives : 3
D6 gives : 1
D6 gives : 1
D6 gives : 2


In [15]:
for i in range(10) :
    D10,value = D10()
    print("D10 gives : " + str(value))

D10 gives : 6
D10 gives : 5
D10 gives : 4
D10 gives : 2
D10 gives : 4
D10 gives : 1
D10 gives : 3
D10 gives : 1
D10 gives : 6
D10 gives : 4


Repeat - but interleaved !! 

In [16]:
D6,_ = diceClass(6,0)
D10,_ = diceClass(6,5)
for i in range(10) :
    D6,value = D6()
    print("D6 gives : " + str(value))
    D10,value = D10()
    print("D10 gives : " + str(value))
    

D6 gives : 4
D10 gives : 6
D6 gives : 6
D10 gives : 5
D6 gives : 2
D10 gives : 4
D6 gives : 4
D10 gives : 2
D6 gives : 6
D10 gives : 4
D6 gives : 3
D10 gives : 1
D6 gives : 3
D10 gives : 3
D6 gives : 1
D10 gives : 1
D6 gives : 1
D10 gives : 6
D6 gives : 2
D10 gives : 4


#### Notes

1) Could you implement the diceClass using pyhton classes ? 
2) Remove the comments from the diceClass function and see if the students recognise the class / object patterns.
3) Do the students see the significance of the interleaving example ? It shows that dice objects are thread safe
4) Can the dice objects be memoised ? Are they pure ? Can the students use their memoise functions from the exercises in the patterns section to memoise the dice objets ?

### End of Point of Departure

## A more declarative Turn

The diceClass can now be used in the turn function.

$Turn : Dice \times Strategy \times N \times N \rightarrow N$

In [49]:
def basic_pigs_turn(dice,strategy,score,opponent_score) :
    running_total = 0
    while strategy(score,opponent_score,running_total) :
        dice,roll = dice()
        if roll == 6 :
            return score
        running_total += roll
    return running_total 

#### Example 

In [50]:
strategy = partial(stick_at_n,target=100,n=10)
dice,_ = diceClass(sides = 6,seed = 0)

In [84]:
basic_pigs_turn(dice,strategy,0,0)

11

In [238]:
value

3

In [210]:
res

<function __main__.f.<locals>.g()>

In [19]:
from functools import partial

In [28]:
a = partial(stick_at_n,n=20)
b = partial(stick_at_n,n=10)

In [121]:
turn(a,b,0,0)

0

In [178]:
def pigs(a,b,target,ascore = 0,bscore = 0) :
    ascore += turn(a,b,ascore,bscore)
    if ascore > target :
        return a
    bscore += turn(b,a,bscore,ascore)
    if bscore > target :
        return b
    return pigs(a,b,target,ascore,bscore)

In [179]:
res = pigs(a,b,100)

In [142]:
res == b

True

In [148]:
def round(a,b,target,rounds) :
    return [game(x,y,target) for x,y in [(a,b)]*rounds]
    

In [149]:
round(a,b,100,20)

[functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=10),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=10),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=10),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
 functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=

In [176]:
def competition(strategies) :
    matches = reduce(lambda a,b : a + b,list(map(lambda y : list(map(lambda x : (y,x) , strategies)),strategies)))
    return {(a,b) : round(a,b,100,20) for a,b in matches}

In [168]:
strategies = [a,b]

In [159]:
matches = list(map(lambda x : map(lambda y : (x,y), strategies),strategies))

In [166]:
from functools import reduce
reduce(lambda a,b : a + b,list(map(lambda y : list(map(lambda x : (y,x) , strategies)),strategies)))

[(1, 1), (1, 2), (2, 1), (2, 2)]

In [177]:
competition([a,b])

{(functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20)): [functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a01101b5da0>, n=20),
  functools.partial(<function stick_at_n at 0x7a

In [6]:
list(map(lambda x : list(map(lambda y : (x,y), X)),X))

[[(1, 1), (1, 2), (1, 3), (1, 4)],
 [(2, 1), (2, 2), (2, 3), (2, 4)],
 [(3, 1), (3, 2), (3, 3), (3, 4)],
 [(4, 1), (4, 2), (4, 3), (4, 4)]]

# Notes

Dice book - need to make a bibtex entry

@BOOK{Knizia2010-ez,
  title     = "Dice games properly explained",
  author    = "Knizia, Reiner",
  publisher = "Blue Terrier Press",
  month     =  jan,
  year      =  2010
}






